In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE175616'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Clinical study of the effect of combined treatment of aspirin and zileuton on biomarkers of tobacco-related carcinogenesis in current smokers"
!Series_summary	"A chemopreventive effect of aspirin (ASA) on lung cancer risk is supported by epidemiologic and preclinical studies. Zileuton, a 5-LOX inhibitor has single agent activity and adds to the activity of NSAIDs in preclinical models of tobacco carcinogenesis   We hypothesized that COX inhibitor + 5-LOX inhibitor may be more effective than placebo in modulating nasal epithelium gene signatures of tobacco exposure and lung cancer. We conducted a randomized, double-blinded study of low dose ASA plus zileuton vs. double placebo in current smokers to compare modulating effects on nasal epithelium gene expression and arachidonic acid (AA) metabolism. Sixty-three participants were randomized to combined treatment of ASA (81 mg QD) and zileuton (Zyflo CR) two 600 mg extended release tablets BID or place

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Checking for gene expression data availability
# Based on the background information and descriptions
is_gene_available = True

# Step 2: Determining variable availability and data type conversion

# COPD trait: Since the background information indicates an effect on COPD,
# but there's no explicit mention of individual sample COPD status in the characteristics, it is considered not available.
trait_row = None

# Age data availability: Age data is found in key 6.
age_row = 6

# Gender data availability: Gender data is found in key 5.
gender_row = 5

# Function to convert age values to continuous data type
def convert_age(value):
    try:
        # Extract the value after the colon and convert to integer
        return int(value.split(':')[-1].strip())
    except ValueError:
        return None

# Function to convert gender values to binary data type
def convert_gender(value):
    gender_value = value.split(':')[-1].strip().lower()
    if gender_value == 'male':
        return 1
    elif gender_value == 'female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE175616', './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Since trait_row is None, no need to extract clinical data for COPD
